머신러닝 등 데이터 분석의 정확도는 분석 데이터의 품질에 의해 좌우된다. 데이터 품질을 높이기 위해서는 누락 데이터, 중복 데이터 등 오류를 수정하고 분석 목적에 맞게 변형해야 한다. 수집한 데이터를 분석에 적합하도록 처리하는 방법을 **'사전 처리(preprocessing)'** 이라고 한다. <br>
데이터프레임에는 원소 데이터 값이 종종 누락된다. 데이터를 파일로 입력할 때 빠트리거나 파일 형식을 변환하는 과정에서 데이터가 소실되는 것이 주요 원인이다. 일반적으로 유효한 데이터 값이 존재하지 않는 누락 데이터를 **'NaN(Not a Number)'**으로 표시한다.

<h3> 누락 데이터 확인 </h3>

Seaborn 라이브러리의 'titanic' 데이터셋을 가져와보고 head() 메소드로 첫 5행을 출력해보자.

In [8]:
import seaborn as sb

# titanic 데이터셋
df = sb.load_dataset('titanic')
print(df.head())

   survived  pclass     sex   age  ...  deck  embark_town  alive  alone
0         0       3    male  22.0  ...   NaN  Southampton     no  False
1         1       1  female  38.0  ...     C    Cherbourg    yes  False
2         1       3  female  26.0  ...   NaN  Southampton    yes   True
3         1       1  female  35.0  ...     C  Southampton    yes  False
4         0       3    male  35.0  ...   NaN  Southampton     no   True

[5 rows x 15 columns]


'deck' 열에 NaN값이 있는 것을 확인할 수 있다.

info() 메소드를 사용해 데이터프레임의 요약 정보를 출력해보자.

In [9]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.6+ KB
None


RangeIndex를 보면 각 열은 891개의 데이터를 가진다. 그리고 'deck' 열에는 203개의 유효한(non-null) 범주형 데이터가 있다. 따라서 'deck' 열에 있는 누락 데이터가 688개인 것을 확인할 수 있다.

counts() 메소드를 이용해도 각 열의 유효한 데이터 개수를 반환해준다. 

In [10]:
print(df.count())

survived       891
pclass         891
sex            891
age            714
sibsp          891
parch          891
fare           891
embarked       889
class          891
who            891
adult_male     891
deck           203
embark_town    889
alive          891
alone          891
dtype: int64


value_counts() 메소드를 이용해서 'deck' 열의 NaN값의 개수를 확인해보자.

In [11]:
nan_deck = df['deck'].value_counts(dropna = False)
print(nan_deck)

NaN    688
C       59
B       47
D       33
E       32
A       15
F       13
G        4
Name: deck, dtype: int64


누락 데이터를 찾는 직접적인 방법에는 **isnull() 메소드**와 **notnull() 메소드**가 있다.

* isnull(): 누락 데이터면 True, 유효한 데이터면 False 반환
* notnull() 누락 데이터면 False, 유효한 데이터면 True 반환

In [12]:
print(df.head())
print(df.head().isnull())
print(df.head().notnull())

   survived  pclass     sex   age  ...  deck  embark_town  alive  alone
0         0       3    male  22.0  ...   NaN  Southampton     no  False
1         1       1  female  38.0  ...     C    Cherbourg    yes  False
2         1       3  female  26.0  ...   NaN  Southampton    yes   True
3         1       1  female  35.0  ...     C  Southampton    yes  False
4         0       3    male  35.0  ...   NaN  Southampton     no   True

[5 rows x 15 columns]
   survived  pclass    sex    age  ...   deck  embark_town  alive  alone
0     False   False  False  False  ...   True        False  False  False
1     False   False  False  False  ...  False        False  False  False
2     False   False  False  False  ...   True        False  False  False
3     False   False  False  False  ...  False        False  False  False
4     False   False  False  False  ...   True        False  False  False

[5 rows x 15 columns]
   survived  pclass   sex   age  ...   deck  embark_town  alive  alone
0      True  

isnull() 메소드를 사용하고 sum(axis = 0) 메소드를 적용하면 각 열의 누락 데이터 개수를 확인할 수 있다.

In [13]:
print(df.head().isnull().sum(axis = 0))

survived       0
pclass         0
sex            0
age            0
sibsp          0
parch          0
fare           0
embarked       0
class          0
who            0
adult_male     0
deck           3
embark_town    0
alive          0
alone          0
dtype: int64


<h3> 누락 데이터 제거 </h3>

누락 데이터가 들어 있는 열 또는 행을 제거하는 방법을 알아보자. <br>
먼저 각 열의 누락 데이터 개수를 알아보자.

In [14]:
missing_df = df.isnull()
for col in missing_df.columns:
  missing_count = missing_df[col].value_counts()
  try:
    print(col, ': ', missing_count[True])
  except:
    print(col, ': ', 0)

survived :  0
pclass :  0
sex :  0
age :  177
sibsp :  0
parch :  0
fare :  0
embarked :  2
class :  0
who :  0
adult_male :  0
deck :  688
embark_town :  2
alive :  0
alone :  0


891명의 승객 중 688명의 'deck' 값이 누락되어 있다. 따라서 누락 데이터를 가진 행을 제거하기 보다 'deck'열을 제거하는 것이 더 좋을 것이다. <br>
**dropna() 메소드**를 사용하면 열 혹은 행을 지울 수 있다. <br>
**thresh 매개변수**를 지정하면 지정한 값 이상의 누락 데이터를 가진 열을 모두 삭제한다.

In [15]:
df_thresh = df.dropna(axis = 1, thresh = 500)
print(df_thresh.columns)

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'embark_town', 'alive',
       'alone'],
      dtype='object')


891명의 승객 중 177명의 'age'값이 누락되어 있다. 승객의 나이가 데이터 분석의 중요한 변수라면 나이 데이터가 없는 승객의 레코드(행)을 제거하는 것이 좋다. <br>
**subset 매개변수**로 확인할 열의 이름들을 리스트로 전달할 수 있다. <br>
**how 매개변수**로 삭제할 방식을 선택할 수 있다. 'any'는 확인할 열 데이터중 하나라도 NaN 값이라면 행을 삭제한다. 'all'은 모든 데이터가 NaN 값일 경우에만 삭제한다.

In [16]:
df_age = df.dropna(subset = ['age'], how = 'any', axis = 0)
print(len(df_age))

714


<h3> 누락 데이터 치환 </h3>

데이터셋의 품질을 높일 목적으로 누락 데이터를 삭제하면 어렵게 수집한 데이터를 활용하지 못하게 된다. 데이터 분석의 정확도는 데이터의 품질 외에도 제공되는 데이터의 양에 의해 상당한 영향을 받는다. 따라서 데이터 중에서 일부가 누락되어 있더라도 나머지 데이터를 최대한 살려 데이터 분석에 활용하는 것이 좋은 결과를 얻는 경우가 많다. <br>
**fillna() 메소드**로 누락 데이터를 대체할 수 있다. 새로운 객체를 반환하므로 원본 객체를 변경하려면 inplace 매개변수를 True로 지정해야한다. <br>
'age'열의 누락 데이터를 'age' 열의 평균값으로 대체해보자.

In [18]:
df = sb.load_dataset('titanic')

print(df['age'].head(10))
print('\n')

mean_age = df['age'].mean(axis = 0)
df['age'].fillna(mean_age, inplace = True)

print(df['age'].head(10))

0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
5     NaN
6    54.0
7     2.0
8    27.0
9    14.0
Name: age, dtype: float64


0    22.000000
1    38.000000
2    26.000000
3    35.000000
4    35.000000
5    29.699118
6    54.000000
7     2.000000
8    27.000000
9    14.000000
Name: age, dtype: float64


승선도시를 나타내는 'embark_town'열에 있는 NaN을 최빈값으로 바꿔보자. 

* idxmax(): 최대값을 가지는 인덱스 레이블을 출력한다.
* idxmin(): 최소값을 가지는 인덱스 레이블을 출력한다.

In [19]:
print(df['embark_town'][825:830])
print('\n')

most_freq = df['embark_town'].value_counts(dropna = True).idxmax()
print(most_freq)
print('\n')

df['embark_town'].fillna(most_freq, inplace = True)

print(df['embark_town'][825:830])

825     Queenstown
826    Southampton
827      Cherbourg
828     Queenstown
829            NaN
Name: embark_town, dtype: object


Southampton


825     Queenstown
826    Southampton
827      Cherbourg
828     Queenstown
829    Southampton
Name: embark_town, dtype: object


* 누락 데이터가 NaN으로 입력되지 않고 ?와 같이 다른 문자로 입력되었다면 replace 메소드를 사용해 Numpy에서 지원하는 np.nan으로 변경해 주는 것이 좋다. <br>
Ex) df.replace('?'. np.nan, inplace = True)

데이터 특성상 서로 이웃하고 있는 데이터끼리 유사성을 가질 가능성이 높다. 이럴 때는 앞이나 뒤에서 이웃하고 있는 값으로 치환해 주는 것이 좋다. NaN값을 NaN이 있는 행의 직전 행에 있는 값으로 바꾸려면 **method 매개변수**를 'ffill'로 지정해야 한다. NaN이 있는 행의 바로 다음 행에 있는 값을 가지고 치환하려면 'bfill'로 지정하면 된다.

In [20]:
df = sb.load_dataset('titanic')

print(df['embark_town'][825:830])
print('\n')

df['embark_town'].fillna(method = 'ffill', inplace = True)
print(df['embark_town'][825:830])

825     Queenstown
826    Southampton
827      Cherbourg
828     Queenstown
829            NaN
Name: embark_town, dtype: object


825     Queenstown
826    Southampton
827      Cherbourg
828     Queenstown
829     Queenstown
Name: embark_town, dtype: object
